In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

In [74]:
class vgg16_mil(nn.Module):
    def __init__(self, concepts_size=10):
        super(vgg16_mil, self).__init__()
        
        # fine-tune the logit layer.
        self.logit = nn.Sequential()
        self.logit.add_module("conv1_1", nn.Conv2d(3, 128, kernel_size=3, stride=1, padding=1))
        self.logit.add_module("fc8_conv", nn.Conv2d(128, concepts_size, kernel_size=1, stride=1, padding=0))
        self.logit.add_module("sigmoid_8", nn.Sigmoid())

        self.pool_mil = nn.MaxPool2d(kernel_size=565, stride=0)

    def forward(self, img, concepts_size=10):
        odds = self.logit(img)
        print odds.shape
        # possibility of each concept given image region j in image i corresponds to word concept
        x = self.pool_mil(odds)
        x = x.squeeze(2).squeeze(2)

        # equation (1) in paper <<From Captions to Visual Concepts and Back>>
        x1 = torch.add(torch.mul(odds.view(x.size(0), concepts_size, -1), -1), 1)
        cumprod = torch.cumprod(x1, 2)

        out = torch.max(x, torch.add(torch.mul(cumprod[:, :, -1], -1), 1))
        return out

In [81]:
i = torch.LongTensor([[0, 0, 0],
                          [8, 1, 2]])
v = torch.FloatTensor([1, 1, 1])
label = torch.sparse.FloatTensor(i, v, torch.Size([1,10])).to_dense()

In [83]:
sum(sum(label))

tensor(3.)

In [76]:
label.shape

torch.Size([1, 10])

In [77]:
md = vgg16_mil()
params = [{'params':md.parameters()}]
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(params, lr=0.001)
image = torch.rand(1, 3, 565,565)




In [84]:
md.zero_grad()
output = md(image)
label.squeeze(0)
loss = criterion(output, label)
loss.backward()
print loss

torch.Size([1, 10, 565, 565])
tensor(1.0133)


In [7]:
if __name__ == '__main__':
    model = vgg16_mil()
    sample = torch.rand(1, 3, 565, 565)
    model.forward(sample)
    # for name, param in model.named_parameters():
    #     print name, param.size()

KeyboardInterrupt: 